# Fine-tune Llama 2 on iSarcasmEval dataset
### This notebook is  inspired by [Fine-tune Llama 2 for sentiment analysis](https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis/notebook) fine-tune-llama-2-for-sentiment-analysis by **Luca Massaron** and [Fine-Tuning LLaMA 2](https://www.datacamp.com/tutorial/fine-tuning-llama-2)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## Preparing Data

In [ ]:
!git clone https://github.com/iabufarha/iSarcasmEval.git

Cloning into 'iSarcasmEval'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 72 (delta 25), reused 32 (delta 6), pack-reused 0
Receiving objects: 100% (72/72), 535.16 KiB | 14.08 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
filename ='/content/iSarcasmEval/train/train.En.csv'
X = pd.read_csv(filename,
                 usecols=["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")

def generate_prompt(data_point):
    return f"""
            Determine the whether the tweet enclosed in square brackets is sarcastic or non-sarcastic,
            and return the answer as the corresponding label "1" for sarcastic or "0" for non-sarcastic.

            [{data_point["tweet"]}] = {data_point["sarcastic"]}
            """.strip()

X = pd.DataFrame(X.apply(generate_prompt, axis=1),
                       columns=["text"])

X = X.sample(frac=1)

train_size = int(len(X)*0.8)
val_size = len(X)-train_size

X_train = X[:train_size]
X_val = X[train_size:]

In [ ]:
X_train

,text
216,Determine the whether the tweet enclosed in sq...
2276,Determine the whether the tweet enclosed in sq...
1505,Determine the whether the tweet enclosed in sq...
1104,Determine the whether the tweet enclosed in sq...
74,Determine the whether the tweet enclosed in sq...
...,...
3166,Determine the whether the tweet enclosed in sq...
156,Determine the whether the tweet enclosed in sq...
287,Determine the whether the tweet enclosed in sq...
1231,Determine the whether the tweet enclosed in sq...


In [ ]:
X_val

,text
787,Determine the whether the tweet enclosed in sq...
2516,Determine the whether the tweet enclosed in sq...
2961,Determine the whether the tweet enclosed in sq...
661,Determine the whether the tweet enclosed in sq...
101,Determine the whether the tweet enclosed in sq...
...,...
1407,Determine the whether the tweet enclosed in sq...
1974,Determine the whether the tweet enclosed in sq...
214,Determine the whether the tweet enclosed in sq...
1526,Determine the whether the tweet enclosed in sq...


In [ ]:
test_filename ='/content/iSarcasmEval/test/task_A_En_test.csv'

X_test = pd.read_csv(test_filename,
                 usecols=["text", "sarcastic"],
                #  names = ["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")

X_test = X_test.rename(columns={'text':'tweet'})

X_test = X_test.sample(frac=1)

def generate_test_prompt(data_point):
    return f"""
            Determine the whether the tweet enclosed in square brackets is sarcastic or non-sarcastic,
            and return the answer as the corresponding label "1" for sarcastic or "0" for non-sarcastic.

            [{data_point["tweet"]}] = """.strip()


y_test = X_test['sarcastic']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [ ]:
X_test

,text
772,Determine the whether the tweet enclosed in sq...
729,Determine the whether the tweet enclosed in sq...
1023,Determine the whether the tweet enclosed in sq...
807,Determine the whether the tweet enclosed in sq...
890,Determine the whether the tweet enclosed in sq...
...,...
620,Determine the whether the tweet enclosed in sq...
1278,Determine the whether the tweet enclosed in sq...
846,Determine the whether the tweet enclosed in sq...
70,Determine the whether the tweet enclosed in sq...


## Functions

In [ ]:
def evaluate(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf",trust_remote_code=True,)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",
                                             device_map=device,
                                              torch_dtype=compute_dtype,
                                              quantization_config=bnb_config,)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "1" in answer:
            y_pred.append(1)
        elif "0" in answer:
            y_pred.append(0)
        else:
            y_pred.append(0)
    return y_pred

## Fine-tuning

In [ ]:
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_val)

In [ ]:
output_dir="trained_weigths"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=4,                       # number of training epochs
    per_device_train_batch_size=4,            # batch size per device during training
    gradient_accumulation_steps=1,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)



In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/2774 [00:00<?, ? examples/s]

Map:   0%|          | 0/694 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,1.091200,0.993958
2,1.101400,0.986780
4,0.968400,1.004224
5,0.913600,1.009059


TrainOutput(global_step=1038, training_loss=0.9256009504285162, metrics={'train_runtime': 2148.1936, 'train_samples_per_second': 7.748, 'train_steps_per_second': 0.483, 'total_flos': 6.17118165983232e+16, 'train_loss': 0.9256009504285162, 'epoch': 5.98})

## Evaluate

In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 1400/1400 [06:55<00:00,  3.37it/s]


In [ ]:
evaluate(y_test.tolist(), y_pred)

Accuracy: 0.831
Accuracy for label 0: 0.841
Accuracy for label 1: 0.770

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.84      0.89      1200
           1       0.45      0.77      0.57       200

    accuracy                           0.83      1400
   macro avg       0.70      0.81      0.73      1400
weighted avg       0.88      0.83      0.85      1400


Confusion Matrix:
[[1009  191]
 [  46  154]]
